# Accessing `/pnfs` on EAF with `pyutils` 

## Introduction 

This turorial is primarily intended to show users how to access files hosted on `/pnfs` (and defined in SAM) while working on EAF. For general information on EAF, see [tutorial](https://github.com/Mu2e/Tutorial/tree/main/EAF). 

`/pnfs` is not mounted on EAF, so the `pyread` module provides functionality to read these files with `xroot` via `mdh`. 

## Table of contents
1. Setup
1. Using `pyread` directly 
1. Using `pyimport`

## 1. Setup

As in [pyutils_basics.ipynb](pyutils_basics.ipynb), we need to add `pyutils` to our Python path.

In [1]:
import sys 
sys.path.append("../../utils/pyutils") 

## 2. Using `pyread` directly 

We can use the `Reader` class with the `use_remote` flag set to `True` to generate an xroot URL for a file and read it with uproot.  

`pyread` relies on `mdh` to generate the URL, and so `Reader` inherits the `location` and `schema` arguments from `mdh`, which are explained below: 

```
$ mdh print-url --help
usage: mdh print-url [-h] [-l LOCATION] [-s SCHEMA] names [names ...]

...

  -l LOCATION, --location LOCATION
                        standard location to use:
                          tape (default), disk, scratch, nersc
  -s SCHEMA, --schema SCHEMA
                        Schema to use in writing the url:
                          path (default),http,root,dcap,sam
```

>**Note**: When using remote files `pyutils`, will set the necessary environment variables (allowing us to leverage Mu2e file handling tools) **once** per process. 


In [2]:
# Import the Reader class
from pyread import Reader

# Define our file name (which is archived in a SAM definition)
file_name = "nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root"

# Initialise the Reader with the appropriate arguments
reader = Reader(
    use_remote = True,
    location = "disk", # This file is located on persistent, so we use "disk" 
    verbosity = 2
)

# Retrieve an uproot object for this file
file = reader.read_file(file_name)



[pyutils] ⭐️ Setting up...
[pyutils] ✅ Ready
[pyread] ⭐️ Opening remote file: nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root
[pyread] ⭐️ Created file path: root://fndcadoor.fnal.gov:1094/mu2e/persistent/datasets/phy-nts/nts/mu2e/CosmicCRYSignalAllOffSpillTriggered-LH/MDC2020as_best_v1_3_v06_03_00/root/08/90/nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root
[pyread] ✅ Opened root://fndcadoor.fnal.gov:1094/mu2e/persistent/datasets/phy-nts/nts/mu2e/CosmicCRYSignalAllOffSpillTriggered-LH/MDC2020as_best_v1_3_v06_03_00/root/08/90/nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root


The URL has been created and the file has been opened, we can now inspect the file...

In [3]:
file.keys()

['EventNtuple;1',
 'EventNtuple/ntuple;19',
 'EventNtuple/ntuple;18',
 'EventNtuple/version;1',
 'genCountLogger;1',
 'genCountLogger/numEvents;1',
 'genCountLogger/numSubRuns;1']

## 3. Using `pyprocess`

`pyprocess` employs `pyread` as a dependancy, so if we are working with TTrees (`EventNtuple`) we can skip `pyread` and use `pyprocess` to retrieve our data as a structured array. 



In [4]:
# Import the Importer class
from pyprocess import Processor

# Define our file name (which is archived in a SAM definition)
file_name = "nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root"

#  Initialise the Importer with the appropriate arguments
processor = Processor( 
    use_remote = True,
    location = "disk",
    verbosity = 2
)

# Define branches
# Branches can also be defined as a list of a grouped dict of lists, depending on requirements
branches = [
    "event",
    "trk.nactive",
    "trkqual.result",
    "crvcoincs.PEs",
    "trksegs"
]

# Import branches 
data = processor.process_data(
    file_name = "nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root",
    # branches = "*" # read all branches
    branches = branches
)

[pyprocess] ⭐️ Initialised Processor:
	path = 'EventNtuple/ntuple'
	use_remote = True
	location = disk
	schema = root
	verbosity=2
[pyread] ⭐️ Opening remote file: nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root
[pyread] ⭐️ Created file path: root://fndcadoor.fnal.gov:1094/mu2e/persistent/datasets/phy-nts/nts/mu2e/CosmicCRYSignalAllOffSpillTriggered-LH/MDC2020as_best_v1_3_v06_03_00/root/08/90/nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root
[pyread] ✅ Opened root://fndcadoor.fnal.gov:1094/mu2e/persistent/datasets/phy-nts/nts/mu2e/CosmicCRYSignalAllOffSpillTriggered-LH/MDC2020as_best_v1_3_v06_03_00/root/08/90/nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root
[pyimport] 👀 Accessing branches in tree:
name                 | typename                 | interpretation                                                                               

We now have an awkward array called `data` which can be used for analysis. 